Autorzy: Diana Misiaczyńska, Emilia Myrta

Celem ćwiczenia było zapoznanie się ze **STRIPS** (Stanford Research Institute Problem Solver), czyli językiem formalnym używanym do wyrażania problemów planowania.

Wszystkie nasze dziedziny oraz problemy planowania wybierałśmy z tych zaproponowanych na [githubie](https://github.com/primaryobjects/strips) przez Kory Becker (z niewielkimi różnicami).

W naszej definicji dziedzin i problemów skorzystałyśmy z niektórych gotowych klas z AIPython, m.in. `stripsProblem.py`, `stripsForwardPlanner.py` oraz `searchMPP.py`.

Import bibliotek i potrzebnych modułów:

In [230]:
from stripsProblem import *
from stripsForwardPlanner import Forward_STRIPS
from searchMPP import SearcherMPP
import time

Za pomocą funkcji `get_actions_from_path(path)` wydobywamy listę akcji z obiektu `Path`, który reprezentuje znalezione rozwiązanie problemu planowania.

In [231]:
def get_actions_from_path(path):
    actions = []
    current = path
    while current.arc is not None:
        actions.append(current.arc.action)
        current = current.initial
    actions.reverse()
    return actions

Poniższa funkcja rozwiązuje zadany problem oraz dwa podcele dla podajnej dziedziny. Oprócz tego liczy czas zarówno dla rozwiązania z heurystyką, jak i bez.

In [232]:
def solutions(domain, initial_state, substate1, substate2, goal_state, heur, heuristic = False):
  start = time.time()
  problem1 = Planning_problem(domain, initial_state, substate1)
  solution1 = SearcherMPP(Forward_STRIPS(problem1)).search()
  problem2 = Planning_problem(domain, substate1, substate2)
  solution2 = SearcherMPP(Forward_STRIPS(problem2)).search()
  problem3 = Planning_problem(domain, substate2, goal_state)
  if(heuristic):
    print("-----------------------------------------")
    print("Uruchamiam planowanie z heurystyką...\n")
    print("-----------------------------------------")
    solution = SearcherMPP(Forward_STRIPS(problem3, heur)).search()
    end = time.time()
    if solution:
        actions = get_actions_from_path(solution1) + get_actions_from_path(solution2) + get_actions_from_path(solution)
        print("Znalezione działania:")
        for i, action in enumerate(actions, 1):
            print(f"{i}. {action}")
        print(f"\nCzas rozwiązania: {end - start:.4f} sekundy")
        print(f"Liczba akcji: {len(actions)}")
    else:
        print("Nie znaleziono rozwiązania.")
  else:
    print("-----------------------------------------")
    print("Uruchamiam planowanie bez heurystyki...\n")
    print("-----------------------------------------")
    solution = SearcherMPP(Forward_STRIPS(problem3)).search()
    end = time.time()
    if solution:
      actions = get_actions_from_path(solution1) + get_actions_from_path(solution2) + get_actions_from_path(solution)
      print("Znalezione działania:")
      for i, action in enumerate(actions, 1):
          print(f"{i}. {action}")
      print(f"\nCzas rozwiązania: {end - start:.4f} sekundy")
      print(f"Liczba akcji: {len(actions)}")
    else:
        print("Nie znaleziono rozwiązania.")

# Problem 1 - Blocks World

Blocks World to jedna z popularniejszych dziedzin planowania. Składa się ona z różynch klocków, które leżą na stole. W naszym przykładzie dziedzina zawierała trzy bloki: `'a'`, `'b'` oraz `'c'`, a także trzy różne miejsca na stole: `'t1'`, `'t2'` i `'t3'`. Tylko jeden blok może być przestawiany równocześnie.


Wybrana przez nas heurystyka opisuje ilość klocków na niepoprawnych miejscach:

In [257]:
def h_blocks_distance(state, goal):
    return sum(1 for key in goal if state.get(key) != goal[key])

In [258]:
blocks = {'a', 'b', 'c'}
tables = {'t1', 't2', 't3'}
all_places = blocks | tables
boolean = {True, False}


def on(x): return f"{x}_on"
def clear(x): return f"clear_{x}"


actions = set()

Możliwymi do wykonania akcjami jest przestawianie każdego z bloków `move_{b}_from_{x}_to_{y}` (na których nie leży inny klocek) na inne miejsce - inny klocek lub na stół.

In [259]:
for b in blocks:
    for x in all_places:
        for y in all_places:
            if x != y and b != x and b != y:
                actions.add(Strips(
                    name=f"move_{b}_from_{x}_to_{y}",
                    preconds={on(b): x, clear(b): True, clear(y): True},
                    effects={on(b): y, clear(x): True, clear(y): False}
                ))

feature_domain_dict = {on(b): all_places - {b} for b in blocks}
feature_domain_dict.update({clear(p): boolean for p in all_places})
blocks_domain = STRIPS_domain(feature_domain_dict, actions)


W stanie początkowym na miejscu `'t1'` na stole leżą następujące klocki tworzące wieżę: `'t1'` -> `'c'` -> `'b'` -> `'a'`.

Naszym celem jest ułożenie identycznej wieży, lecz na miejscu `'t2'`.

In [260]:
initial_state = {
    on('c'): 't1',
    on('b'): 'c',
    on('a'): 'b',
    clear('a'): True,
    clear('b'): False,
    clear('c'): False,
    clear('t1'): False,
    clear('t2'): True,
    clear('t3'): True
}

In [261]:
substate1 = {
    on('a'): 't1',
    on('b'): 't2',
    on('c'): 't3',
    clear('a'): True,
    clear('b'): True,
    clear('c'): True,
    clear('t1'): False,
    clear('t2'): False,
    clear('t3'): False
}

In [262]:
substate2 = {
    on('a'): 'b',
    on('b'): 'c',
    on('c'): 't3',
    clear('a'): True,
    clear('b'): False,
    clear('c'): False,
    clear('t1'): True,
    clear('t2'): True,
    clear('t3'): False
}

In [263]:
goal_state = {
    on('a'): 'b',
    on('b'): 'c',
    on('c'): 't2'
}

In [264]:
solutions(blocks_domain, initial_state, substate1, substate2, goal_state, h_blocks_distance, True)

Solution: {'c_on': 't1', 'b_on': 'c', 'a_on': 'b', 'clear_a': True, 'clear_b': False, 'clear_c': False, 'clear_t1': False, 'clear_t2': True, 'clear_t3': True}
   --move_a_from_b_to_t3--> {'c_on': 't1', 'b_on': 'c', 'a_on': 't3', 'clear_a': True, 'clear_b': True, 'clear_c': False, 'clear_t1': False, 'clear_t2': True, 'clear_t3': False}
   --move_b_from_c_to_t2--> {'c_on': 't1', 'b_on': 't2', 'a_on': 't3', 'clear_a': True, 'clear_b': True, 'clear_c': True, 'clear_t1': False, 'clear_t2': False, 'clear_t3': False}
   --move_a_from_t3_to_b--> {'c_on': 't1', 'b_on': 't2', 'a_on': 'b', 'clear_a': True, 'clear_b': False, 'clear_c': True, 'clear_t1': False, 'clear_t2': False, 'clear_t3': True}
   --move_c_from_t1_to_t3--> {'c_on': 't3', 'b_on': 't2', 'a_on': 'b', 'clear_a': True, 'clear_b': False, 'clear_c': True, 'clear_t1': True, 'clear_t2': False, 'clear_t3': False}
   --move_a_from_b_to_t1--> {'c_on': 't3', 'b_on': 't2', 'a_on': 't1', 'clear_a': True, 'clear_b': True, 'clear_c': True, 'clea

In [265]:
solutions(blocks_domain, initial_state, substate1, substate2, goal_state, None, False)

Solution: {'c_on': 't1', 'b_on': 'c', 'a_on': 'b', 'clear_a': True, 'clear_b': False, 'clear_c': False, 'clear_t1': False, 'clear_t2': True, 'clear_t3': True}
   --move_a_from_b_to_t3--> {'c_on': 't1', 'b_on': 'c', 'a_on': 't3', 'clear_a': True, 'clear_b': True, 'clear_c': False, 'clear_t1': False, 'clear_t2': True, 'clear_t3': False}
   --move_b_from_c_to_t2--> {'c_on': 't1', 'b_on': 't2', 'a_on': 't3', 'clear_a': True, 'clear_b': True, 'clear_c': True, 'clear_t1': False, 'clear_t2': False, 'clear_t3': False}
   --move_a_from_t3_to_b--> {'c_on': 't1', 'b_on': 't2', 'a_on': 'b', 'clear_a': True, 'clear_b': False, 'clear_c': True, 'clear_t1': False, 'clear_t2': False, 'clear_t3': True}
   --move_c_from_t1_to_t3--> {'c_on': 't3', 'b_on': 't2', 'a_on': 'b', 'clear_a': True, 'clear_b': False, 'clear_c': True, 'clear_t1': True, 'clear_t2': False, 'clear_t3': False}
   --move_a_from_b_to_t1--> {'c_on': 't3', 'b_on': 't2', 'a_on': 't1', 'clear_a': True, 'clear_b': True, 'clear_c': True, 'clea

# Problem 2 - Magic World

Przykład Magic World modeluje scenariusz, w którym gracz musi pokonać potwory i zebrać magiczne składniki z różnych lokalizacji, by stworzyć kulę ognia `has-fireball`. Problem wymaga sekwencji działań takich jak ruch, walka, otwieranie skrzyń i łączenie elementów.

Heurystyka opisuje liczbę brakujących cech do celu.

In [242]:
def h_fireball(state, goal):
    cost = 0

    if not state.get('has-fire', False):
        cost += 3
        if not state.get('open_box1', False):
            cost += 1
        if state.get('guarded_river', True):
            cost += 1
        if state.get('at_npc') != 'river':
            cost += 1

    if not state.get('has-earth', False):
        cost += 3
        if not state.get('open_box2', False):
            cost += 1
        if state.get('guarded_cave', True):
            cost += 1
        if state.get('at_npc') != 'cave':
            cost += 1

    if not state.get('has-fireball', False):
        if state.get('has-fire', False) and state.get('has-earth', False):
            cost += 1
        else:
            cost += 0

    return cost


In [243]:
locations = {'town', 'field', 'river', 'cave'}
monsters = {'ogre', 'dragon'}
chests = {'box1', 'box2'}
elements = {'reddust', 'browndust'}
players = {'npc'}

feature_domain = {
    'at_npc': locations,
    'at_ogre': locations,
    'at_dragon': locations,
    'guarded_river': boolean,
    'guarded_cave': boolean,
    'at_box1': locations,
    'at_box2': locations,
    'open_box1': boolean,
    'open_box2': boolean,
    'in_reddust': chests,
    'in_browndust': chests,
    'empty_box1': boolean,
    'empty_box2': boolean,
    'has-fire': boolean,
    'has-earth': boolean,
    'has-fireball': boolean
}

actions = set()

Wśród możliwych akcji znajdują się: przesunięcie gracza z jednego miejsca do innego `move_npc_from_{l1}_to_{l2}`, pokonanie potwora `attack_{monster}_from_{l1}`, otworzenie skrzynki `open_box` oraz zebranie potrzebnych elementów (`collect_fire`, `collect_earth`) do zbudowania kuli ognia `build_fireball`.

In [244]:
borders =  {
    'town': ['field'],
    'field': ['town', 'river'],
    'river': ['field', 'cave'],
    'cave': ['river']
}

for l1 in borders:
    for l2 in borders[l1]:
        actions.add(Strips(
            name=f"move_npc_from_{l1}_to_{l2}",
            preconds={'at_npc': l1, f'guarded_{l2}': False},
            effects={'at_npc': l2}
        ))

for monster, loc in [('ogre', 'river'), ('dragon', 'cave')]:
    for l1 in borders[loc]:
        actions.add(Strips(
            name=f"attack_{monster}_from_{l1}",
            preconds={'at_npc': l1, f'at_{monster}': loc, f'guarded_{loc}': True},
            effects={f'at_{monster}': None, f'guarded_{loc}': False}
        ))

In [245]:
actions.add(Strips(
    name="open_box1",
    preconds={'at_npc': 'river', 'at_box1': 'river', 'open_box1': False},
    effects={'open_box1': True}
))
actions.add(Strips(
    name="open_box2",
    preconds={'at_npc': 'cave', 'at_box2': 'cave', 'open_box2': False},
    effects={'open_box2': True}
))

actions.add(Strips(
    name="collect_fire",
    preconds={'at_npc': 'river', 'at_box1': 'river', 'open_box1': True,
              'in_reddust': 'box1', 'empty_box1': False},
    effects={'has-fire': True, 'empty_box1': True}
))

actions.add(Strips(
    name="collect_earth",
    preconds={'at_npc': 'cave', 'at_box2': 'cave', 'open_box2': True,
              'in_browndust': 'box2', 'empty_box2': False},
    effects={'has-earth': True, 'empty_box2': True}
))

actions.add(Strips(
    name="build_fireball",
    preconds={'has-fire': True, 'has-earth': True},
    effects={'has-fireball': True, 'has-fire': False, 'has-earth': False}
))

In [246]:
magic_domain = STRIPS_domain(feature_domain, actions)

W początkowym stanie gracz znajduje się w mieście `'town'`, rzeka `'river'` oraz jaskinia `'cave'` są strzeżone przez potwory oraz gracz nie posiada żadnych elementów do zbudowania kuli ognia.

Naszym stanem docelowym jest posiadanie kuli ognia `has-fireball`.

In [247]:
initial_state = {
    'at_npc': 'town',
    'at_ogre': 'river',
    'at_dragon': 'cave',
    'guarded_river': True,
    'guarded_cave': True,
    'guarded_field': False,
    'guarded_town': False,
    'at_box1': 'river',
    'at_box2': 'cave',
    'open_box1': False,
    'open_box2': False,
    'in_reddust': 'box1',
    'in_browndust': 'box2',
    'empty_box1': False,
    'empty_box2': False,
    'has-fire': False,
    'has-earth': False,
    'has-fireball': False
}

substate1 = {
    'at_npc': 'town',
    'at_ogre': None,
    'at_dragon': 'cave',
    'guarded_river': False,
    'guarded_cave': True,
    'guarded_field': False,
    'guarded_town': False,
    'at_box1': 'river',
    'at_box2': 'cave',
    'open_box1': True,
    'open_box2': False,
    'in_reddust': 'box1',
    'in_browndust': 'box2',
    'empty_box1': True,
    'empty_box2': False,
    'has-fire': True,
    'has-earth': False,
    'has-fireball': False
}

substate2 = {
    'at_npc': 'town',
    'at_ogre': None,
    'at_dragon': None,
    'guarded_river': False,
    'guarded_cave': False,
    'guarded_field': False,
    'guarded_town': False,
    'at_box1': 'river',
    'at_box2': 'cave',
    'open_box1': True,
    'open_box2': True,
    'in_reddust': 'box1',
    'in_browndust': 'box2',
    'empty_box1': True,
    'empty_box2': True,
    'has-fire': True,
    'has-earth': True,
    'has-fireball': False
}

goal_state = {
    'has-fireball': True
}

In [248]:
solutions(magic_domain, initial_state, substate1, substate2, goal_state, h_fireball, True)

Solution: {'at_npc': 'town', 'at_ogre': 'river', 'at_dragon': 'cave', 'guarded_river': True, 'guarded_cave': True, 'guarded_field': False, 'guarded_town': False, 'at_box1': 'river', 'at_box2': 'cave', 'open_box1': False, 'open_box2': False, 'in_reddust': 'box1', 'in_browndust': 'box2', 'empty_box1': False, 'empty_box2': False, 'has-fire': False, 'has-earth': False, 'has-fireball': False}
   --move_npc_from_town_to_field--> {'at_npc': 'field', 'at_ogre': 'river', 'at_dragon': 'cave', 'guarded_river': True, 'guarded_cave': True, 'guarded_field': False, 'guarded_town': False, 'at_box1': 'river', 'at_box2': 'cave', 'open_box1': False, 'open_box2': False, 'in_reddust': 'box1', 'in_browndust': 'box2', 'empty_box1': False, 'empty_box2': False, 'has-fire': False, 'has-earth': False, 'has-fireball': False}
   --attack_ogre_from_field--> {'at_npc': 'field', 'at_ogre': None, 'at_dragon': 'cave', 'guarded_river': False, 'guarded_cave': True, 'guarded_field': False, 'guarded_town': False, 'at_box1'

In [249]:
solutions(magic_domain, initial_state, substate1, substate2, goal_state, None, False)

Solution: {'at_npc': 'town', 'at_ogre': 'river', 'at_dragon': 'cave', 'guarded_river': True, 'guarded_cave': True, 'guarded_field': False, 'guarded_town': False, 'at_box1': 'river', 'at_box2': 'cave', 'open_box1': False, 'open_box2': False, 'in_reddust': 'box1', 'in_browndust': 'box2', 'empty_box1': False, 'empty_box2': False, 'has-fire': False, 'has-earth': False, 'has-fireball': False}
   --move_npc_from_town_to_field--> {'at_npc': 'field', 'at_ogre': 'river', 'at_dragon': 'cave', 'guarded_river': True, 'guarded_cave': True, 'guarded_field': False, 'guarded_town': False, 'at_box1': 'river', 'at_box2': 'cave', 'open_box1': False, 'open_box2': False, 'in_reddust': 'box1', 'in_browndust': 'box2', 'empty_box1': False, 'empty_box2': False, 'has-fire': False, 'has-earth': False, 'has-fireball': False}
   --attack_ogre_from_field--> {'at_npc': 'field', 'at_ogre': None, 'at_dragon': 'cave', 'guarded_river': False, 'guarded_cave': True, 'guarded_field': False, 'guarded_town': False, 'at_box1'

# Problem 3 - Birthday Dinner

Przykład Birthday Dinner demonstruje planowanie działań z efektami ubocznymi – niektóre akcje, choć niezbędne, naruszają inne warunki wymagane do osiągnięcia celu. Planowanie polega na znalezieniu odpowiedniej kolejności operacji, np. wyniesienia śmieci, posprzątania, ugotowania obiadu i zapakowania prezentu.

Heurystyka opisuje liczbę niespełnionych celów.

In [250]:
def h_birthday(state, goal):
    cost = 0

    if state.get('garbage_x', True):
        cost += 1

    if not state.get('dinner_x', False):
        if not state.get('clean_x', False):
            cost += 2
        else:
            cost += 1

    if not state.get('present_x', False):
        if not state.get('quiet_x', False):
            cost += 2
        else:
            cost += 1

    return cost


In [251]:
objects = {'x'}

feature_domain = {
    'clean_x': boolean,
    'quiet_x': boolean,
    'garbage_x': boolean,
    'dinner_x': boolean,
    'present_x': boolean
}

actions = set()

Możliwymi akcjami jest gotowanie `cook_x`, pakowanie prezentu `wrap_x`, wynoszenie śmieci `carry_x`, wynioszenie śmieci przy pomocy wózka `dolly_x` oraz posprzątanie `clean_up_x`.

In [252]:
actions.add(Strips(
    name="cook_x",
    preconds={'clean_x': True},
    effects={'dinner_x': True}
))


actions.add(Strips(
    name="wrap_x",
    preconds={'quiet_x': True},
    effects={'present_x': True}
))


actions.add(Strips(
    name="carry_x",
    preconds={'garbage_x': True},
    effects={'garbage_x': False, 'clean_x': False}
))


actions.add(Strips(
    name="dolly_x",
    preconds={'garbage_x': True},
    effects={'garbage_x': False, 'quiet_x': False}
))

actions.add(Strips(
    name="clean_up_x",
    preconds={'clean_x': False},
    effects={'clean_x': True}
))

In [253]:
birthday_domain = STRIPS_domain(feature_domain, actions)

W naszym stanie początkowym śmieci są niewynisione, jest brudno, cicho, kolacje nie jest przygotowana oraz prezent nie jest zapakowany.

Naszym celem jest wyniesienie śmieci, przygotowanie kolacji oraz zapakowanie prezentu.

In [254]:
initial_state = {
    'garbage_x': True,
    'clean_x': False,
    'quiet_x': True,
    'dinner_x': False,
    'present_x': False
}

substate1 = {
    'garbage_x': True,
    'clean_x': True,
    'quiet_x': True,
    'dinner_x': False,
    'present_x': False
}

substate2 = {
    'garbage_x': False,
    'clean_x': True,
    'quiet_x': True,
    'dinner_x': True,
    'present_x': False
}

goal_state = {
    'garbage_x': False,
    'dinner_x': True,
    'present_x': True
}

birthday_problem = Planning_problem(birthday_domain, initial_state, goal_state)

In [255]:
solutions(birthday_domain, initial_state, substate1, substate2, goal_state, h_birthday, True)

Solution: {'garbage_x': True, 'clean_x': False, 'quiet_x': True, 'dinner_x': False, 'present_x': False}
   --clean_up_x--> {'garbage_x': True, 'clean_x': True, 'quiet_x': True, 'dinner_x': False, 'present_x': False} (cost: 1)
 5 paths have been expanded and 7 paths remain in the frontier
Solution: {'garbage_x': True, 'clean_x': True, 'quiet_x': True, 'dinner_x': False, 'present_x': False}
   --carry_x--> {'garbage_x': False, 'clean_x': False, 'quiet_x': True, 'dinner_x': False, 'present_x': False}
   --clean_up_x--> {'garbage_x': False, 'clean_x': True, 'quiet_x': True, 'dinner_x': False, 'present_x': False}
   --cook_x--> {'garbage_x': False, 'clean_x': True, 'quiet_x': True, 'dinner_x': True, 'present_x': False} (cost: 3)
 14 paths have been expanded and 12 paths remain in the frontier
-----------------------------------------
Uruchamiam planowanie z heurystyką...

-----------------------------------------
Solution: {'garbage_x': False, 'clean_x': True, 'quiet_x': True, 'dinner_x': T

In [256]:
solutions(birthday_domain, initial_state, substate1, substate2, goal_state, None, False)

Solution: {'garbage_x': True, 'clean_x': False, 'quiet_x': True, 'dinner_x': False, 'present_x': False}
   --clean_up_x--> {'garbage_x': True, 'clean_x': True, 'quiet_x': True, 'dinner_x': False, 'present_x': False} (cost: 1)
 5 paths have been expanded and 7 paths remain in the frontier
Solution: {'garbage_x': True, 'clean_x': True, 'quiet_x': True, 'dinner_x': False, 'present_x': False}
   --carry_x--> {'garbage_x': False, 'clean_x': False, 'quiet_x': True, 'dinner_x': False, 'present_x': False}
   --clean_up_x--> {'garbage_x': False, 'clean_x': True, 'quiet_x': True, 'dinner_x': False, 'present_x': False}
   --cook_x--> {'garbage_x': False, 'clean_x': True, 'quiet_x': True, 'dinner_x': True, 'present_x': False} (cost: 3)
 14 paths have been expanded and 12 paths remain in the frontier
-----------------------------------------
Uruchamiam planowanie bez heurystyki...

-----------------------------------------
Solution: {'garbage_x': False, 'clean_x': True, 'quiet_x': True, 'dinner_x':